In [2]:
%load_ext autoreload
%autoreload 2

from incomer.data import DataManager
import pandas as pd

filename = '../data/EURUSD60.csv'
cycles = [5, 20, 50, 100, 200, 600, 1000]
num_bars = 500
forward_bars = 48
lookbacks = 120
valid_split = 0.8

dm = DataManager(filename, cycles, num_bars=num_bars)

xtrain, ytrain, xtest, ytest = dm.create_train_data(valid_split, lookbacks, forward_bars)

print(xtrain.shape)
print(xtest.shape)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


  0%|          | 0/119 [00:00<?, ?it/s]c:\users\admin\desktop\incomer\incomer\incomer\data.py:76: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"C{c}{lookback}"] = df[f"C{c}"].shift(lookback)
c:\users\admin\desktop\incomer\incomer\incomer\data.py:71: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"hightail{lookback}"] = df.hightail.shift(lookback)
c:\users\admin\desktop\incomer\incomer\incomer\data.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times

(1036, 120, 10)
(76, 120, 10)



c:\users\admin\desktop\incomer\incomer\incomer\data.py:133: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ytrain = np.array([*y_over, *ytrain])


In [ ]:
len(dm.op)

In [5]:
from incomer.models.transformer import TradeTransformer

trader = TradeTransformer()
trader.build_model(
    input_shape=xtrain.shape[1:],
    head_size=64,
    num_heads=16,
    ff_dim=4,
    num_transformer_blocks=32,
    mlp_units=[128, 64],
    mlp_dropout=0.3,
    dropout=0.3,
)
# trader.model.summary()

In [6]:
trader.train(
    xtrain,
    ytrain,
    xtest,
    dm.body,
    dm.op,
    epochs=999999999,
    batch_size=16,
    valid_splits=0.8,
    save_path="data/eurusd_h1/best",
)
